In [3]:
import numpy as np

# if your model is saved as only a .h5 file
MODEL_LOCATION ='3-conv-128-nodes-1-dense-model image classification.h5'


In [4]:
if MODEL_LOCATION!='': #if your model is saved as a .h5 file only
    from keras.models import load_model
    model = load_model(MODEL_LOCATION,compile=False) #load the model
    print("loaded model from MODEL_LOCATION")

2023-07-28 02:41:56.750742: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


loaded model from MODEL_LOCATION


In [7]:
def convert_h5_to_aws(loaded_model):
    """
    given a pre-trained keras model, this function converts it to a TF protobuf format
    and saves it in the file structure which aws expects
    """  
    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    
    # This is the file structure which AWS expects. Cannot be changed. 
    model_version = '1'
    export_dir = 'export/Servo1/' + model_version
    
    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)
    
    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})
    
    from keras import backend as K
    with K.get_session() as sess:
        # Save the meta graph and variables
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()
    
    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model.tar1.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
        
convert_h5_to_aws(model)


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo1/1/saved_model.pb


In [8]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar1.gz', key_prefix='model')

In [9]:
print("Bucket name is:")
sagemaker_session.default_bucket()

Bucket name is:


'sagemaker-us-east-1-075921637588'

In [10]:
!touch train1.py

In [11]:
import boto3, re
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

# Create a Sagemaker model (see AWS console>SageMaker>Models)
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train1.py')

In [12]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

-----!

In [13]:
# What is our endpoint called?
endpoint = predictor.endpoint
endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sagemaker-tensorflow-serving-2023-07-28-02-46-11-647'

In [14]:
import boto3

# Replace 'endpoint_name' with the name of your SageMaker endpoint
endpoint_name = 'sagemaker-tensorflow-serving-2023-07-26-11-59-14-066'

# Create a boto3 session to interact with SageMaker
boto3_session = boto3.Session()

# Use the boto3 session to create the SageMaker client
sagemaker_client = boto3_session.client('sagemaker')

try:
    # Describe the SageMaker endpoint to get information, including the region
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    
    # Get the endpoint ARN from the response
    endpoint_arn = response['EndpointArn']
    
    # Extract the region from the ARN
    region = endpoint_arn.split(':')[3]
    
    print(f"The region of the endpoint '{endpoint_name}' is: {region}")

except sagemaker_client.exceptions.ResourceNotFound:
    print(f"Endpoint '{endpoint_name}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")


The region of the endpoint 'sagemaker-tensorflow-serving-2023-07-26-11-59-14-066' is: us-east-1
